# Offline Feature Extraction

In [110]:
import numpy as np
import scipy.io
import scipy.integrate
import pylab
import matplotlib.pyplot as plt
import mne
from mne.time_frequency import psd_multitaper
import time
 
%matplotlib inline

mne.set_log_level('WARNING')

## Set Parameters

In [111]:
freq_s = 250    # [Hz] Sampling Frequency
trial_len = 1.    # [sec] Length of a single trial
rest_len = 0.5    # [sec] Length of rest before the trial
n_channels = 3    # set numpy of channels to use from raw eeg matrix
alpha_range = [8, 14]    # [Hz] range of alpha frequency
alpha_subbands_frq = np.array([np.arange(8,12),np.arange(10,14)]).T.astype(float)    # Sub bands of alpha
print('Alpha frequency range: {}\nAlpha sub bands:\n{}'.format(alpha_range, alpha_subbands_frq))

Alpha frequency range: [8, 14]
Alpha sub bands:
[[  8.  10.]
 [  9.  11.]
 [ 10.  12.]
 [ 11.  13.]]


## Read Raw Data

In [112]:
# Load Raw EEG
raw_fname = '/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Code/eeg_feature_extraction/exp_1-raw.fif'
raw = mne.io.read_raw_fif(raw_fname, add_eeg_ref=False).load_data()

# Remove EOG channels (channels 4-6)
raw.drop_channels(['eog_4','eog_5', 'eog_6'])

<Raw  |  exp_1-raw.fif, n_channels x n_times : 3 x 552452 (2209.8 sec), ~12.7 MB, data loaded>

#### Remove Artifacts

In [113]:
# Load data containing artifacts
data_path = "/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Code/eeg_feature_extraction/B01T.mat"
mat = scipy.io.loadmat(data_path)['data']

# Get artifacts
exp_arr = mat[0,0]
# explanation:   exp_arr[0][0][data]
exp_dict = {'X': exp_arr[0][0][0].T,
            'trial': exp_arr[0][0][1],
            'y': exp_arr[0][0][2],
            'fs': exp_arr[0][0][3].flatten(),
             'classes': exp_arr[0][0][4].flatten(),
             'artifacts': exp_arr[0][0][5],
             'gender': exp_arr[0][0][6],
             'age': exp_arr[0][0][7].flatten()
             }

# Remove from 'trials' and 'y'
artifact_idx = exp_dict['artifacts'].nonzero()[0]
exp_dict['trial'] = np.delete(exp_dict['trial'], artifact_idx)
exp_dict['y'] = np.delete(exp_dict['y'], artifact_idx)

# Functions

## Function: power_difference( raw,  dt_rest,  dt_trial )
Calculate the power difference of the rest and trial phase of an eeg signal :
- Calculate PSD for rest and trial
- Integrate over PSD of rest and trial
- Calculate difference between power rest and trial


#### Inpunt
* raw: band filtered mne raw object
* rest_len: [sec] length of rest interval
* trial_len: [sec] length of trial interval


#### Outpunt
* delta_power: np.array of delta of power values of each channel between rest and trial phase in raw object




In [114]:
def power_difference(raw, rest_len, trial_len):
    # Parameters
    f_min, f_max = 5, 14
    # PSD
    psds_rest, freqs_test = psd_multitaper(raw, low_bias=True, tmin=0.0, tmax=rest_len,
                                           fmin=f_min, fmax=f_max, proj=True)
    psds_trial, freqs_trial = psd_multitaper(raw, low_bias=True, tmin=rest_len, tmax=rest_len+trial_len,
                                             fmin=f_min, fmax=f_max, proj=True)
    # Power
    power_arr_rest = np.zeros(len(psds_rest))    
    power_arr_trial = np.zeros(len(psds_trial))
    for i in range(len(psds_trial)):    # iterate through channels
        power_arr_rest[i] = scipy.integrate.simps(psds_rest[i])
        power_arr_trial[i] = scipy.integrate.simps(psds_trial[i])
    
    # Power difference
    delta_powers = power_arr_rest - power_arr_trial
    
    return delta_powers

# Create Feature Matrix

#### X.shape = (n_samples, n_subbands*n_channels)

In [115]:
# Init X
n_trials   = len(exp_dict['trial'])
n_subbands = len(alpha_subbands_frq)
n_channels = len(raw.ch_names)
X = np.zeros((n_trials, n_subbands * n_channels))
# Init y
y = np.zeros(len(exp_dict['y']))

print("X.shape: ", X.shape)
print("y.shape: ", y.shape)

X.shape:  (102, 12)
y.shape:  (102,)


#### Feature Matrix X

In [116]:
# Calculate Features
filter_len = int((trial_len + rest_len) * freq_s)

t0 = time.time()

for t, trial_idx in enumerate(exp_dict['trial']):    # t: iterate over trials
    # crop relevant interval  
    raw_temp = raw.copy()
    trial_t = trial_idx/freq_s
    try:
        raw_trial_crop = raw_temp.crop(tmin=trial_t-rest_len, tmax=trial_t+trial_len)
        
        raw_trial_sub_list = [[] for _ in range(len(alpha_subbands_frq))]
        for s, subband in enumerate(alpha_subbands_frq):    # s: iterate over subbands
            raw_temp = raw_trial_crop.copy()
            # filter subbands
            raw_alpha_sub = raw_temp.filter(l_freq=subband[0], h_freq=subband[1], 
                                            filter_length=filter_len, l_trans_bandwidth='auto', # filter_length='auto'
                                            h_trans_bandwidth='auto', n_jobs=1, method='fir', 
                                            iir_params=None, phase='zero', fir_window='hamming')
            # calc power
            # delta_powers: for each channel one power difference
            delta_powers = power_difference(raw, rest_len, trial_len)

            # add to feature matrix
            for c, d_power in enumerate(delta_powers):    # c: iterate over channels
                X[t, s + (c*n_subbands)] = d_power
                
            # add label to y
            y[t] = exp_dict['y'][t]
            
    except:
        print("Did not add to X: trial_idx={}, trial_t={}, tmin={}, tmax={}".format(
                    trial_idx, trial_t, trial_t-rest_len, trial_t+trial_len))
    
    
    
print("Creating Feature Matrix takes {:0.2f} seconds".format(time.time()-t0))

Did not add to X: trial_idx=553654, trial_t=2214.616, tmin=2214.116, tmax=2215.616
Did not add to X: trial_idx=560414, trial_t=2241.656, tmin=2241.156, tmax=2242.656
Did not add to X: trial_idx=567567, trial_t=2270.268, tmin=2269.768, tmax=2271.268
Did not add to X: trial_idx=572489, trial_t=2289.956, tmin=2289.456, tmax=2290.956
Did not add to X: trial_idx=574536, trial_t=2298.144, tmin=2297.644, tmax=2299.144
Did not add to X: trial_idx=579284, trial_t=2317.136, tmin=2316.636, tmax=2318.136
Did not add to X: trial_idx=581622, trial_t=2326.488, tmin=2325.988, tmax=2327.488
Did not add to X: trial_idx=584072, trial_t=2336.288, tmin=2335.788, tmax=2337.288
Did not add to X: trial_idx=588636, trial_t=2354.544, tmin=2354.044, tmax=2355.544
Did not add to X: trial_idx=590826, trial_t=2363.304, tmin=2362.804, tmax=2364.304
Did not add to X: trial_idx=593107, trial_t=2372.428, tmin=2371.928, tmax=2373.428
Did not add to X: trial_idx=601938, trial_t=2407.752, tmin=2407.252, tmax=2408.752
Crea

In [139]:
# Delete samples with 0 in lebel vect
zero_idx_list = np.where(y == 0)[0]
X = np.delete(X, zero_idx_list, axis=0)
y = np.delete(y, zero_idx_list)

In [141]:
X.shape

(90, 12)

In [142]:
y.shape

(90,)

In [145]:
np.save('X_feat_mat.npy', X)

In [146]:
np.save('y_label_vect.npy', y)